<a href="https://colab.research.google.com/github/annisafadila/1320123001-data-mining/blob/main/Tugas12_DataMining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Part 1 — Problem Understanding (20%)**

1. Konsekuensi False Negative (FN)

* FN = pasien sebenarnya sakit tapi model memprediksi sehat.

* Konsekuensi: pasien terkena penyakit serius tidak mendapat perawatan tepat waktu → risiko memburuk dan kematian. Ini masalah keselamatan nyawa sehingga sangat kritis. (lihat penjelasan imbalanced: kelas minoritas penting).

2. Konsekuensi False Positive (FP)

* FP = pasien sehat tapi diprediksi sakit.

* Konsekuensi: dirujuk untuk tes lanjutan yang mahal (menambah biaya rumah sakit & beban pasien), kecemasan pasien, dan potensi overload layanan. Dalam kasus ini tiap FP = biaya Rp 5.000.000 (asumsi tugas).

3. Metrik paling penting: Recall (Sensitivitas)

Kenapa? Karena FN membahayakan nyawa — kita butuh memastikan sebanyak mungkin pasien sakit terdeteksi. Recall = TP / (TP + FN) mengukur proporsi pasien sakit yang berhasil tertangkap. Untuk data imbalanced, recall (dan AUC-PR) lebih relevan daripada accuracy.

**Part 2 — Analytical Strategy (30%)**

**Step 1: Data Preparation**

Teknik handling imbalance yang digunakan:

1. Cost-sensitive learning (class weights) — beri bobot lebih besar pada kelas “sakit” saat pelatihan.

2. SMOTE (Synthetic Minority Oversampling) — buat sampel sintetis untuk kelas minoritas jika diperlukan validasi/augmentation.

3. (Opsional) Ensemble + undersampling stratified untuk robustness.

Mengapa:

* Cost-sensitive langsung memaksa model “menghargai” kesalahan FN lebih tinggi tanpa membuat banyak duplikasi data (mengurangi overfit). SMOTE membantu model belajar variasi kelas minoritas saat jumlah sample riil terbatas. Penjelasan dan trade-off ada di materi Imbalanced Data.

**Step 2: Model Selection**

Rekomendasi model:

1. XGBoost / LightGBM (gradient boosting) atau Random Forest dengan class_weight atau scale_pos_weight.

2. Alternatif lebih sederhana: Logistic Regression dengan class weights (untuk interpretabilitas).

Mengapa:

* Tree ensembles handal untuk fitur campuran (numerik + kategorikal), tahan terhadap outlier, dan mendukung weighting/imbalanced. Juga mudah melakukan probability calibration dan threshold tuning.

**Step 3: Evaluation Framework**

* Metrik evaluasi utama: Recall (Sensitivitas) — prioritas untuk keselamatan pasien.

* Metrik sekunder: Precision (untuk mengontrol jumlah FP) dan F1-score (harmonik recall & precision). Juga AUC-PR untuk gambaran keseluruhan pada data imbalanced.

* Business metric: Total biaya kesalahan (contoh formula):
Total Cost = FP_count * Cost_per_FP + FN_count * Cost_per_FN
(Cost_per_FP = Rp 5.000.000; Cost_per_FN bisa dipandang sebagai biaya sangat besar—baik moneter maupun non-moneter seperti kehilangan nyawa — sebaiknya didefinisikan oleh manajemen/risk team).

**Part 3 — Threshold Tuning (30%)**

Diberikan confusion matrix:
|               | Prediksi: Sakit | Prediksi: Sehat |
| ------------- | --------------: | --------------: |
| Aktual: Sakit |        400 (TP) |        100 (FN) |
| Aktual: Sehat |        200 (FP) |      9,300 (TN) |

Total pasien = 400 + 100 + 200 + 9,300 = 10,000.

Hitungan (tahap demi tahap)

1. Accuracy
Accuracy = (TP + TN) / Total
= (400 + 9,300) / 10,000
= 9,700 / 10,000 = 0.97 → 97%

2. Precision (untuk kelas "sakit")
Precision = TP / (TP + FP)
= 400 / (400 + 200) = 400 / 600 = 0.6666667 → 66.67%

3. Recall (Sensitivitas)
Recall = TP / (TP + FN)
= 400 / (400 + 100) = 400 / 500 = 0.8 → 80%

4. F1-Score
F1 = 2 * (Precision * Recall) / (Precision + Recall)
Precision = 2/3 (≈0.6666667), Recall = 0.8
F1 = 2 * (0.6666667 * 0.8) / (0.6666667 + 0.8)
= 2 * 0.5333333 / 1.4666667
= 1.0666667 / 1.4666667 = 0.7272727 → 72.73%

**Business Analysis (angka konkret)**

* Berapa pasien sakit yang terlewat (FN)? = 100 pasien (ini adalah kasus paling berbahaya).

* Berapa pasien sehat yang dirujuk tes lanjutan (FP)? = 200 pasien.

* Total biaya kesalahan (dengan asumsi FP = Rp 5 juta; =
Cost_FP = 200 × Rp 5.000.000 = Rp 1.000.000.000 → Rp 1.000.000.000.
(Catatan: FN = risiko nyawa — biaya non-moneter besar.

**Part 4 — Optimization Recommendation (20%)**

Syarat tim medis: "Kami ingin minimal 90% pasien sakit terdeteksi." → artinya Recall ≥ 90%.
1. Trade-off yang harus diterima

* Untuk meningkatkan recall dari 80% → 90%, kita harus menurunkan threshold atau membuat model lebih “sensitif” sehingga lebih banyak prediksi “sakit”.

* Akibatnya FP akan naik (lebih banyak pasien sehat dirujuk → biaya tes meningkat). Jadi trade-off: lebih sedikit FN (baik untuk keselamatan) vs lebih banyak FP (lebih mahal).

2. Strategy rekomendasi (praktis & sederhana)

* Gunakan dua-tahap (two-stage) screening:

1. Stage 1 (screening sensitif): model high-recall (turunkan threshold) — menahan FN rendah.

2. Stage 2 (konfirmasi): untuk mereka yang positif di stage 1, jalankan tes lanjutan/algoritma kedua yang lebih spesifik (mis. pemeriksaan klinis atau model kedua yang dioptimalkan untuk precision) sehingga mengurangi FP akhir.

* Gabungkan cost-sensitive learning (beri bobot lebih tinggi untuk kelas sakit) saat training.

* Threshold tuning pada validation set untuk target recall 90% sambil memonitor biaya FP pada data validasi.

* Monitoring & feedback klinis: lakukan audit klinis terhadap kasus yang salah untuk perbaikan berkelanjutan.

3. Threshold bagaimana yang optimal? (contoh angka ilustratif)

* Contoh ilustrasi (bukan hasil sebenarnya, hanya gambaran trade-off):

* Saat ini recall = 80% (TP=400, FN=100). Untuk mencapai recall 90% pada 500 pasien sakit, kita butuh TP = 450 (FN = 50).

* Misal penurunan threshold menaikkan FP dari 200 → 600 (hipotesis). Maka biaya FP baru = 600 × Rp 5.000.000 = Rp 3.000.000.000.

* Bandingkan: kita mengurangi FN dari 100 → 50 (50 pasien yang sebelumnya terlewat sekarang tertangkap). Apakah rumah sakit bersedia menanggung tambahan biaya Rp 2 miliar (selisih) untuk menyelamatkan 50 pasien? Keputusan ini harus dibuat oleh manajemen/komite etika/keuangan.

* Praktik: tidak beri angka threshold mutlak (mis. 0.3) tanpa uji validasi: lakukan threshold sweep pada validation set dan pilih threshold dengan Recall ≥ 90% dan biaya/FP yang masih dapat diterima oleh pihak rumah sakit. Selain itu gunakan two-stage untuk menekan biaya tambahan.